In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


In [3]:
# Cargamos los datos
agua = pd.read_csv(r'C:\Users\usuario\Desktop\Universidad\TERCERO\Primer_Cuatri\Hackaton\Reto\agua.csv')
embalses = pd.read_csv(r'C:\Users\usuario\Desktop\Universidad\TERCERO\Primer_Cuatri\Hackaton\Reto\embalsesUTF8.csv')
listado = pd.read_csv(r'C:\Users\usuario\Desktop\Universidad\TERCERO\Primer_Cuatri\Hackaton\Reto\listadoUTF8-3.tsv', delimiter='\t')

In [18]:
print(f'Longitud de agua: {len(agua)}, longitud de embalses: {len(embalses)}, longitud de listado: {len(listado)}')

Longitud de agua: 683447, longitud de embalses: 401, longitud de listado: 367


In [28]:
agua.head()

,FECHA,AGUA_ACTUAL,ID
0,04/09/12 00:00,56.0,18
1,23/10/12 00:00,50.0,18
2,27/11/12 00:00,55.0,18
3,22/01/91 00:00,53.0,18
4,21/05/91 00:00,52.0,18


In [54]:
embalses.head()

,ID,AMBITO_NOMBRE,EMBALSE_NOMBRE,AGUA_TOTAL,ELECTRICO_FLAG
0,1,GUADALQUIVIR,"BREÑA, LA",103.0,0
1,3,GUADALQUIVIR,"FERNANDINA, LA",247.0,0
2,5,GUADALQUIVIR,PUEBLA DE CAZALLA,87.0,0
3,6,GUADALQUIVIR,PEDRO MARÍN,19.0,0
4,9,CUENCA MEDITERRÁNEA ANDALUZA,"VIÑUELA, LA",170.0,0


In [55]:
'''
Mediante un inner join en SQL, conseguimos unir con las claves primarias 'EMBALSE_NOMBRE' y 'NOMBRE' las tablas 'embalses' y 'listado'.
De esta forma, conseguimos un total de 224 embalses distintos en la tabla obtenida de implementar la relación.
A continuación, mostramos lo que se ha realizado en Python para obtener, de los que no se relacionaron de primeras, 48 nuevos embalses
que no coincidían directamente en las claves primarias.

Se ha procedido a sustituir las tildes por vocales sin tilde, los determinantes por nada y los símbolos por nada.
'''
df_outer = pd.merge(embalses, listado, left_on='EMBALSE_NOMBRE', right_on='NOMBRE', how='outer')
df_left = df_outer[df_outer['EMBALSE_NOMBRE'].isna()].copy()
df_right = df_outer[df_outer['NOMBRE'].isna()].copy()
df_left['LeftJoin'] = df_left['NOMBRE']
df_right['RightJoin'] = df_right['EMBALSE_NOMBRE']
tildes = ['Á', 'É', 'Í', 'Ó', 'Ú', 'á', 'é', 'í', 'ó', 'ú']
vocales = ['A', 'E', 'I', 'O', 'U', 'a', 'e', 'i', 'o', 'u']
for i in range(10):
    df_left['LeftJoin'] = df_left['LeftJoin'].str.upper()
    df_right['RightJoin'] = df_right['RightJoin'].str.upper()
    df_left['LeftJoin'] = df_left['LeftJoin'].str.replace(tildes[i], vocales[i])
    df_right['RightJoin'] = df_right['RightJoin'].str.replace(tildes[i], vocales[i])

determinantes = ['El', 'La', 'Los', 'Las', 'LA', 'EL', 'LOS', 'LAS']
for i in range(8):
    df_left['LeftJoin'] = df_left['LeftJoin'].str.replace(determinantes[i], '')
    df_right['RightJoin'] = df_right['RightJoin'].str.replace(determinantes[i], '')

simbolos = [',', '-', '.', '(', ')']
for i in range(5):
    df_left['LeftJoin'] = df_left['LeftJoin'].str.replace(simbolos[i], '')
    df_right['RightJoin'] = df_right['RightJoin'].str.replace(simbolos[i], '')

df_left = df_left.drop_duplicates(subset='LeftJoin')
df_right = df_right.drop_duplicates(subset='RightJoin')

df_left = df_left.drop(columns=['ID', 'AMBITO_NOMBRE', 'EMBALSE_NOMBRE', 'AGUA_TOTAL', 'ELECTRICO_FLAG'])
df_right = df_right.drop(columns=['CODIGO', 'NOMBRE', 'EMBALSE', 'X', 'Y', 'DEMARC', 'CAUCE',	'GOOGLE',	'OPENSTREETMAP',	'WIKIDATA',	'PROVINCIA',	'CCAA',	'TIPO',	'COTA_CORON',	'ALT_CIMIEN',	'INFORME'])

nuevos_casos = pd.merge(df_right, df_left, left_on='LeftJoin', right_on='RightJoin', how='inner')
print(nuevos_casos)

nuevos_casos.to_csv('embalsesExtra.csv', index=False)


KeyError: 'RightJoin'

In [108]:
nuevos_casos.head()

,CODIGO,NOMBRE,EMBALSE,X,Y,DEMARC,CAUCE,GOOGLE,OPENSTREETMAP,WIKIDATA,...,COTA_CORON,ALT_CIMIEN,INFORME,LeftJoin,ID,AMBITO_NOMBRE,EMBALSE_NOMBRE,AGUA_TOTAL,ELECTRICO_FLAG,RightJoin
0,2370021.0,AGUEDA,AGUEDA,"40,529781939","-6,48161367799997",DUERO,RÍO ÁGUEDA,NaN,NaN,NaN,...,"662,12","37,6",https://sig.mapama.gob.es/WebServices/clientew...,AGUEDA,453.0,DUERO,ÁGUEDA,24.0,0.0,AGUEDA
1,4060045.0,"AGUIJON, EL","AGUIJON, EL","38,473447859","-6,92088528799997",GUADIANA,RIO ALCARRACHE,NaN,NaN,NaN,...,NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,AGUIJON,114.0,GUADIANA,"AGUIJÓN, EL",11.0,0.0,AGUIJON
2,8160006.0,ALARCON,Alarcón Reservoir,"39,5643548690001","-2,11225016699996",JUCAR,RÍO JÚCAR,https://www.google.com/search?kgmid=/g/120z_bn_,NaN,https://www.wikidata.org/wiki/Q3206248,...,814,67,https://sig.mapama.gob.es/WebServices/clientew...,ARCON,646.0,JÚCAR,ALARCÓN,1118.0,0.0,ARCON
3,2370034.0,ALDEADAVILA,Aldeadávila Reservoir,"41,211678448","-6,68578589399993",DUERO,RÍO DUERO,NaN,NaN,https://www.wikidata.org/wiki/Q43551868,...,NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,ALDEADAVI,122.0,DUERO,ALDEADÁVILA,115.0,1.0,ALDEADAVI
4,5110039.0,ALMODOVAR,Embalse de Almodóvar,"36,1536149680001","-5,64973106899993",GUADALETE Y BARBATE,RÍO ALMODÓVAR,NaN,NaN,https://www.wikidata.org/wiki/Q5369429,...,"107,2",40,https://sig.mapama.gob.es/WebServices/clientew...,ALMODOVAR,240.0,GUADALETE-BARBATE,ALMODÓVAR,6.0,0.0,ALMODOVAR


In [7]:
df = pd.read_csv('embalsesExtra.csv')
df.drop(columns=['RightJoin'], inplace=True)
#df.head()

,CODIGO,NOMBRE,EMBALSE,X,Y,DEMARC,CAUCE,GOOGLE,OPENSTREETMAP,WIKIDATA,...,COTA_CORON,ALT_CIMIEN,INFORME,LeftJoin,ID,AMBITO_NOMBRE,EMBALSE_NOMBRE,AGUA_TOTAL,ELECTRICO_FLAG,RightJoin
0,2370021.0,AGUEDA,AGUEDA,"40,529781939","-6,48161367799997",DUERO,RÍO ÁGUEDA,NaN,NaN,NaN,...,"662,12","37,6",https://sig.mapama.gob.es/WebServices/clientew...,AGUEDA,453.0,DUERO,ÁGUEDA,24.0,0.0,AGUEDA
1,4060045.0,"AGUIJON, EL","AGUIJON, EL","38,473447859","-6,92088528799997",GUADIANA,RIO ALCARRACHE,NaN,NaN,NaN,...,NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,AGUIJON,114.0,GUADIANA,"AGUIJÓN, EL",11.0,0.0,AGUIJON
2,8160006.0,ALARCON,Alarcón Reservoir,"39,5643548690001","-2,11225016699996",JUCAR,RÍO JÚCAR,https://www.google.com/search?kgmid=/g/120z_bn_,NaN,https://www.wikidata.org/wiki/Q3206248,...,814,67,https://sig.mapama.gob.es/WebServices/clientew...,ARCON,646.0,JÚCAR,ALARCÓN,1118.0,0.0,ARCON
3,2370034.0,ALDEADAVILA,Aldeadávila Reservoir,"41,211678448","-6,68578589399993",DUERO,RÍO DUERO,NaN,NaN,https://www.wikidata.org/wiki/Q43551868,...,NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,ALDEADAVI,122.0,DUERO,ALDEADÁVILA,115.0,1.0,ALDEADAVI
4,5110039.0,ALMODOVAR,Embalse de Almodóvar,"36,1536149680001","-5,64973106899993",GUADALETE Y BARBATE,RÍO ALMODÓVAR,NaN,NaN,https://www.wikidata.org/wiki/Q5369429,...,"107,2",40,https://sig.mapama.gob.es/WebServices/clientew...,ALMODOVAR,240.0,GUADALETE-BARBATE,ALMODÓVAR,6.0,0.0,ALMODOVAR


In [26]:
todo = pd.read_csv(r'C:\Users\usuario\Desktop\Universidad\TERCERO\Primer_Cuatri\Hackaton\export.csv')
#todo['ID'] = todo['ID'].str.split('.', expand=True)
# Vamos a hacer un split de la columna ID asignando el primer elemento de la lista generada
# por el split a la columna ID
todo['ID'] = todo['ID'].astype(str).str.split('.').str[0]
print(todo.head())


    ID      AMBITO_NOMBRE EMBALSE_NOMBRE  AGUA_TOTAL  ELECTRICO_FLAG  \
0  170           GUADIANA       BOQUERÓN         6.0             0.0   
1  462               EBRO          BÚBAL        64.0             0.0   
2  255               TAJO        BUENDÍA      1705.0             0.0   
3  725               TAJO      CASTREJÓN        44.0             1.0   
4  571  GUADALETE-BARBATE        CELEMÍN        45.0             0.0   

      CODIGO              EMBALSE                 X                   Y  \
0  4060008.0             BOQUERON  39,1511076260001   -6,41827824299997   
1  9220016.0      Búbal reservoir       42,68098159  -0,315761256999963   
2  3190007.0    Buendía Reservoir  40,3992227420001   -2,78382182099995   
3  3450033.0  Castrejón Reservoir      39,833565075   -4,29494820899993   
4  5110032.0   Embalse de Celemín      36,299322019   -5,77443215499994   

                DEMARC  ...  \
0             GUADIANA  ...   
1                 EBRO  ...   
2                 TAJO 

In [27]:
df = pd.merge(todo, agua, on='ID', how='inner')
print(df.head())

ValueError: You are trying to merge on object and int64 columns for key 'ID'. If you wish to proceed you should use pd.concat

In [29]:
agua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683447 entries, 0 to 683446
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   FECHA        683447 non-null  object 
 1   AGUA_ACTUAL  683445 non-null  float64
 2   ID           683447 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.6+ MB


In [30]:
todo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              272 non-null    object 
 1   AMBITO_NOMBRE   272 non-null    object 
 2   EMBALSE_NOMBRE  272 non-null    object 
 3   AGUA_TOTAL      271 non-null    float64
 4   ELECTRICO_FLAG  272 non-null    float64
 5   CODIGO          272 non-null    float64
 6   EMBALSE         272 non-null    object 
 7   X               272 non-null    object 
 8   Y               272 non-null    object 
 9   DEMARC          272 non-null    object 
 10  CAUCE           269 non-null    object 
 11  GOOGLE          98 non-null     object 
 12  OPENSTREETMAP   29 non-null     object 
 13  WIKIDATA        152 non-null    object 
 14  PROVINCIA       272 non-null    object 
 15  CCAA            272 non-null    object 
 16  TIPO            272 non-null    object 
 17  COTA_CORON      131 non-null    obj

In [33]:
todo['ID'] = todo['ID'].astype(str)
agua['ID'] = agua['ID'].astype(str)

df = pd.merge(todo, agua, on='ID', how='inner')

AttributeError: 'DataFrame' object has no attribute 'type'